# Day 12

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_aoc_problem(day, year=2023):
    url = f"https://adventofcode.com/{year}/day/{day}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # raises an exception for HTTP errors

        soup = BeautifulSoup(response.text, 'html.parser')
        
        problem_text = soup.find('article').get_text()
        return problem_text
    except Exception as e:
        return f"Error fetching problem: {e}"

day = 12
problem_prompt = get_aoc_problem(day)
print(problem_prompt)

In [ ]:
try:
    # Open and read the file
    with open('input.txt', 'r') as file:
        lines = file.read().strip().split('\n')

    # Print each line
    for line in lines:
        print(line)

except FileNotFoundError:
    # Specific exception for a clearer error message
    print('Input file not found.')

except Exception as e:
    # Catch other exceptions and print the error
    print(f'An error occurred: {e}')


### Utility functions

In [ ]:
def process_line(line, part_2 = False):
    '''
    input : string (line of the input)

    output : int (number of possible different configurations for that line)
    '''

    row_data, digit_data = line.split()[0], line.split()[1]

    index_list = []

    for i, row_char in enumerate(row_data):
        if row_char == '?':
            index_list.append(i)

    if not part_2:
        num_of_configurations = check_configurations(row_data, index_list, 0, digit_data)
    else:
        # print('PART 2')
        num_of_configurations = check_configurations_2(row_data, index_list, 0, digit_data)

    return num_of_configurations



In [ ]:
def check_configurations(row_data, index_list, current_index, digit_data):
    # print(f'Called check_configurations \n with: (row_data, index_list, current_index, digit_data) = {(row_data, index_list, current_index, digit_data)}')
    
    if current_index < len(index_list):
        # havent reached the end yet:

        index_to_change = index_list[current_index]

        # check for errors
        assert row_data[index_to_change] == '?'

        if index_to_change > 0:
            prefix = row_data[:index_to_change]
        else: 
            prefix = ""
        
        if index_to_change+1 < len(row_data):
            suffix = row_data[index_to_change+1:]
        else:
            suffix = ""
        
        row_data_A = prefix + '.' + suffix
        row_data_B = prefix + '#' + suffix

        configuration_count = 0

        configuration_count += check_configurations(row_data_A, index_list, current_index + 1, digit_data)
        configuration_count += check_configurations(row_data_B, index_list, current_index + 1, digit_data)

        return configuration_count

    else:
        # reached the end, time to validate:

        # should have no unknown spots at this point
        assert '?' not in row_data

        return 1 if get_digit_data(row_data) == digit_data else 0

In [ ]:
def get_digit_data(row_str):
    current_length = 0
    out_str_nums = []
    for i in range(len(row_str)):
        if row_str[i] == '#':
            current_length += 1
        elif current_length > 0:
            out_str_nums.append(str(current_length))
            current_length = 0
    
    if current_length > 0:
            out_str_nums.append(str(current_length))
      
    return ','.join(out_str_nums)

### Testing get_digit_data

In [ ]:
test_lines = [
    "#.#.###",
    ".#...#....###.",
    ".#.###.#.######",
    "####.#...#...",
    "#....######..#####.",
    ".###.##....#"
]

for test_line in test_lines:
    print(get_digit_data(test_line))

In [ ]:
def get_partial_digit_data(row_str):
    current_length = 0
    out_str_nums = []

    #everything to the left of the ?
    left_part = row_str.split("?")[0]

    for i in range(len(left_part)):
        if left_part[i] == '#':
            current_length += 1
        elif current_length > 0:
            out_str_nums.append(str(current_length))
            current_length = 0
    
    if current_length > 0:
            out_str_nums.append(str(current_length))
      
    return ','.join(out_str_nums)

In [ ]:
def check_configurations_2(row_data, index_list, current_index, digit_data):
    
    if current_index < len(index_list):

        # print('-'*40)
        # print(f'depth = {current_index}')
        # print(f'(row_data, index_list[current_index], digit_data) = {(row_data, index_list[current_index], digit_data)}')
        # print('-'*40)

        # havent reached the end yet:

        index_to_change = index_list[current_index]

        # check for errors
        assert row_data[index_to_change] == '?'

        if index_to_change > 0:
            prefix = row_data[:index_to_change]
        else: 
            prefix = ""
        
        if index_to_change+1 < len(row_data):
            suffix = row_data[index_to_change+1:]
        else:
            suffix = ""
        
        row_data_A = prefix + '.' + suffix
        row_data_B = prefix + '#' + suffix

        configuration_count = 0

        # check if the current row_data is valid:
        if validate_incomplete_row(row_data_A, digit_data):
            # print(f'(d = {current_index}) still valid if . added at position {index_list[current_index]}')
            configuration_count += check_configurations_2(row_data_A, index_list, current_index + 1, digit_data)
        else:
            # print(f'(d = {current_index}) no longer valid if . added at position {index_list[current_index]}') 
            # return configuration_count
            pass

        # check if the current row_data is valid:
        if validate_incomplete_row(row_data_B, digit_data):
            # print(f'(d = {current_index}) still valid if # added at position {index_list[current_index]}')
            configuration_count += check_configurations_2(row_data_B, index_list, current_index + 1, digit_data)
        else:
            # print(f'(d = {current_index}) no longer valid if # added at position {index_list[current_index]}') 
            # return configuration_count
            pass

        return configuration_count

    else:
        # print('reached the end, time to validate...')

        # should have no unknown spots at this point
        assert '?' not in row_data

        return 1 if get_digit_data(row_data) == digit_data else 0

In [ ]:
def validate_incomplete_row(row_data, digit_data):
    '''returns true if the current row_data is possible given the digit_data'''

    partial_digit_data = get_partial_digit_data(row_data)

    digit_list = digit_data.split(',')
    partial_digit_list = partial_digit_data.split(',')

    # print(digit_list, partial_digit_list)

    if len(digit_list) < len(partial_digit_list):
        return False

    if len(partial_digit_data) == 0:
        return True

    for i in range(len(partial_digit_list)):

        if partial_digit_list[i] == digit_list[i]:
            continue
        elif int(partial_digit_list[i]) > int(digit_list[i]):
            return False
    
    return True
    

In [ ]:
print(validate_incomplete_row('.#.###.#.######','1,3,1,6'))
print(validate_incomplete_row('.#.###.#.##.###','1,3,1,6'))

### Part 1

In [ ]:
import tqdm
from tqdm import tqdm

total_sum = 0

for line in tqdm(lines):
    total_sum += process_line(line)

print(total_sum)

### Part 2

In [ ]:
def part_2_line_process(line):
    left_line, right_line = line.split()[0], line.split()[1]

    left_og = left_line
    right_og = right_line

    for _ in range(4):
        left_line += '?' + left_og

    for _ in range(4):
        right_line += ',' + right_og

    return left_line + ' ' + right_line

In [ ]:
# total_new_sum = 0

with open('output.txt', 'r') as f:
    output_data = f.read()
    output_lines = output_data.split()
    num_of_lines = len(output_lines)



with open('output.txt', 'w') as f:

    for line in tqdm(lines[num_of_lines:min(num_of_lines+10, len(lines))]):
        new_line = part_2_line_process(line)
        sum_from_line = process_line(new_line, part_2 = True)
        # total_new_sum += sum_from_line
        print('Calculated a line!')
        print(f'Number of combinations = {sum_from_line}')
        f.write(f'Number of combinations = {sum_from_line}')
    
    # print(f'Total sum of configurations = {total_new_sum}')
    

In [ ]:
with open('output.txt', 'w') as f:
    pass

In [ ]:
test_row_1 = '?#.###.#.######'
test_row_2 = '.#.###.?.######'
test_row_3 = '.#.###.#.##?###'


print(get_partial_digit_data(test_row_1))
print(get_partial_digit_data(test_row_2))
print(get_partial_digit_data(test_row_3))